# **Validation Suite**

## Purpose and Setup
This notebook will allow you to test the output from WRF-GHG against surface, upper air, and satelite observations.

The next cell imports modules needed to work properly

In [1]:
import wrf
import numpy as np
import pandas as pd
from datetime import datetime as dt
from netCDF4 import Dataset #type: ignore
from termcolor import cprint
from numpy import unravel_index
import collections.abc as c
import metpy as mp

### *Base class*

This class is used for all the following classes in order to give some basic location data to each class.

In [2]:
class Base_point:
    '''
    Base_point: parent class for validation. Sets location name.
    '''
    def __init__(self: object, loc: str, **kwargs) -> None:
        self.loc = loc

### *Upper air (UA) class*

This class is used to set the attributes for any data objects that cares about upper air data. This is usually the WRF data that you're testing or the radiosonde data you use for validation. It inherits from the base class.

In [3]:
class UA_point(Base_point):
    def __init__(self: object, loc: str, **kwargs) -> None:
        super().__init__(loc, **kwargs)
    def __eq__(self: object, other: object) -> bool:
        try:
            results: np.ndarray[bool] = np.empty(5, bool)
            results[0] = np.allclose(self.p, other.p, atol=10.)
            results[1] = np.allclose(self.t, other.t, atol=1.)
            results[2] = np.allclose(self.td, other.td, atol=1.)
            results[3] = np.allclose(self.wdir, other.wdir, atol=5.)
            results[4] = np.allclose(self.wspd, other.wspd, atol=1.)
            result: bool = results.all()
        except AttributeError:
            if isinstance(self, WRF_point):
                result: bool = other.__eq__(self)
            else:
                raise NotImplementedError('Compairison not implemented')
        finally:
            return result

### *Satelite (Sat) class*

This class is used to set the attributes for any data objects that cares about satelite data. This is usually the WRF data or TROPOMI or OCO-2 data. It inherits from the base class. **Note**: The respective classes for TROPOMI and OCO-2 are still in development (9-9-2024)

In [4]:
class Sat_point(Base_point):
    def __init__(self: object, loc: str, **kwargs) -> None:
        super().__init__(loc, **kwargs)
    def sat_loc(self: object, ulat: float, ulon: float, lats: c.Iterable[float], lons: c.Iterable[float]) -> tuple[int, int] | int:
        R = 6371000
        lat1 = np.radians(ulat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats-ulat)
        delta_lon = np.radians(lons-ulon)
        a = (np.sin(delta_lat/2))*(np.sin(delta_lat/2))+(np.cos(lat1))*(np.cos(lat2))*(np.sin(delta_lon/2))*(np.sin(delta_lon/2))
        c = 2*np.arctan2(np.sqrt(a),np.sqrt(1-a))
        d = R*c
        if d.ndim == 1:
            return d.argmin()
        else:
            x, y = unravel_index(d.argmin(),d.shape)
            return x,y
    def __eq__(self: object, other: object) -> bool:
        result: bool | None = None
        try:
            if isinstance(self, Tropomi_point) or (isinstance(self, WRF_point) and isinstance(other, Tropomi_point)):
                results: np.ndarray[bool] = np.empty(2, bool)
                results[0] = np.abs(self.xch4 - other.xch4) <= 1.e-1
                results[1] = np.abs(self.xco - other.xco) <= 1.e-1
                result = results.all()
            #! elif for OCO-2 here
        except AttributeError:
            if isinstance(self, WRF_point):
                result = other.__eq__(self)
            else:
                result = NotImplemented
        else:
            if result is None:
                if isinstance(self,WRF_point):
                    result = other.__eq__(self)
                else:
                    raise NotImplementedError('Compairison not implemented')
            else:
                pass
        finally:
            return result

### *Surface class*

This class is used to set the attirbutes for any data object that cares about surface data. This is usually the WRF data or the ASOS observation data. It inherits from the base class.

In [5]:
class Surface_point(Base_point):
    def __init__(self: object, loc: str, **kwargs) -> None:
        super().__init__(loc, **kwargs)
    def __eq__(self: object, other: object) -> bool:
        try:
            results: np.ndarray[bool] = np.empty(5, bool)
            results[0] = abs(self.T2 - other.T2) <= 0.5
            results[1] = abs(self.td2 - other.td2) <= 0.5
            results[2] = abs(self.slp - other.p) <= 3.0
            results[3] = abs(self.wspd10 - other.wspd10) <= 0.2
            results[4] = abs(self.wdir10 - other.wdir10) <= 5
            result: bool = results.all()
        except AttributeError:
            if isinstance(self, WRF_point):
                result: bool = other.__eq__(self)
            else:
                raise NotImplementedError('Comparison not implemented')
        finally:
            return result

### *WRF class*

This class ingests the wrfout data that you need to run the validation suite. Since it's what we're testing, it inherits from UA, Sat, and Surface classes.

In [6]:
class WRF_point(Surface_point,Sat_point,UA_point):
    '''
    WRF_point: sets up validation point using WRF data. Reads T2, TD2, SLP, and 10m Wind speed/direction. Inherits from Base_point.
    '''
    def __init__(self: object, wrffile: Dataset, lat: float, lon: float, loc: str, chem: bool | None = None, **kwargs) -> None:
        super().__init__(loc,**kwargs)
        self.lat: float = lat
        self.lon: float = lon
        self.x, self.y = wrf.ll_to_xy(wrffile,self.lat,self.lon)
        self.vars = ['T2', 'td2', 'slp','uvmet10_wspd_wdir','p','temp','td','uvmet_wspd_wdir']
        for var in self.vars:
            if var == 'T2':
                self.T2 = wrf.getvar(wrffile, var, meta=False)[self.y, self.x]
            elif var == 'td2':
                self.td2 = wrf.getvar(wrffile, var, meta=False, units='K')[self.y, self.x]
            elif var == 'slp':
                self.slp = wrf.getvar(wrffile, var, meta=False, units='hPa')[self.y, self.x]
            elif var == 'uvmet10_wspd_wdir':
                self.wspd10, self.wdir10 = wrf.getvar(wrffile, var, meta=False)[:, self.y, self.x]
            elif var == 'p':
                self.p = wrf.getvar(wrffile, var, meta=False, units='hpa')[:, self.y, self.x]
            elif var == 'temp':
                self.t = wrf.getvar(wrffile, var, meta=False)[:, self.y, self.x]
            elif var == 'td':
                self.td = wrf.getvar(wrffile, var, meta=False, units='K')[:, self.y, self.x]
            elif var == 'uvmet_wspd_wdir':
                self.wspd, self.wdir = wrf.getvar(wrffile, var, meta=False)[..., self.y, self.x]
        ## ! Next if/elif block needs to be edited depending on WRF-GHG output structure (include converting units) ! ## 
        self.sfc_pres = wrffile['PSFC'][0, self.y, self.x].tolist() #! may need to calculate or may be in WRF output
        if chem is True:
            self.xch4 = self._extract_ghg(wrffile, 'xch4')
            self.xco = self._extract_ghg(wrffile, 'xco')
            self.xco2 = self._extract_ghg(wrffile, 'xco2')
    def _extract_ghg(self, wrffile: Dataset, chem: str):
        if chem == 'xch4':
            _ant = wrffile['CH4_ANT'][0, :, self.y, self.x]
            _bck = wrffile['CH4_BCK'][0, :, self.y, self.x]
            _tst = wrffile['CH4_TST'][0, :, self.y, self.x]
        elif chem == 'xco2':
            _ant = wrffile['CO2_ANT'][0, :, self.y, self.x]
            _bck = wrffile['CO2_BCK'][0, :, self.y, self.x]
            _tst = wrffile['CO2_TST'][0, :, self.y, self.x]
        elif chem == 'xco':
            _ant = wrffile['CO_ANT'][0, :, self.y, self.x]
            _bck = wrffile['CO_BCK'][0, :, self.y, self.x]
            #_tst = wrffile['CO_BIO'][0, :, self.y, self.x] ##?
            _tst = np.zeros_like(_bck)
        _ghg = _tst + _ant -_bck
        if len(_ghg) == len(self.p):
            pres_bound = np.empty_like(self.p)
            for i, pres in enumerate(self.p):
                if i == 0:
                    pres_bound[i] = self.sfc_pres
                    pres_bound[i+1] = pres_bound[i] + (2*(pres-pres_bound[i]))
                else:
                    try:
                        pres_bound[i+1] = pres_bound[i] + (2*(pres-pres_bound[i]))
                    except IndexError:
                        pass
        p_layer_diff = np.array([pres_bound[i]-pres_bound[i-1] for i in range(1,len(pres_bound))]) #! This may need a value at beginning for xch4[0]
        p_diff = pres_bound[0] - pres_bound[-1]
        return np.sum(_ghg*p_layer_diff)/p_diff
    def __str__(self) -> str:
        return f'{self.loc} WRF Point has a temperature of {self.T2} K, a dewpoint of {self.td2} K, a slp of {self.slp} hPa, and the wind is {self.wspd10} m s^-1 at {self.wdir10} degrees.'

In [7]:
class UObs_point(UA_point):
    def __init__(self, loc, ua_file, wrffile, lat, lon, **kwargs):
        super().__init__(loc, **kwargs)
        x, y = wrf.ll_to_xy(wrffile, lat, lon)
        wrf_p = wrf.getvar(wrffile, 'p', meta=False, units='hPa')[:, y, x]
        data = pd.read_csv(ua_file, na_values='M', parse_dates=['validUTC'], date_format='%Y-%m-%d %H:%M')
        p = data.pressure_mb.to_numpy()
        idx = np.digitize(wrf_p, p)
        data = data.iloc[idx]
        self.p = data.pressure_mb.to_numpy() #* mb == hPa
        self.t = data.tmpc.to_numpy() + 273.15 #* deg C -> K
        self.td = data.dwpc.to_numpy() + 273.15 #* deg C -> K
        self.wdir = data.drct.to_numpy() #* deg
        self.wspd = data.speed_kts.to_numpy() * 0.514444 #* kt -> m s^-1
        del data, p, idx, wrf_p, x, y

In [8]:
lat = 40.87
lon = -72.87

wrf_file = './wrfout/wrfout_d02_2023-07-21_00:00:00'
wrf_ds = Dataset(wrf_file)
loc = 'Brookhaven (KOKX)'
ua_file = '../KOKX_2023072100_2023072112.txt'
wrf_data = WRF_point(wrf_ds, lat, lon, loc)
ua_data = UObs_point(loc, ua_file, wrf_ds, lat, lon)

In [10]:
ua_data == wrf_data

False

In [15]:
np.allclose(ua_data.t, wrf_data.t, atol=25.)

False

In [17]:
print(f'{len(ua_data.t) = }, {len(wrf_data.t) = }')

len(ua_data.t) = 47, len(wrf_data.t) = 47


In [19]:
print(f'{ua_data.t[20] = }, {wrf_data.t[20] = }')

ua_data.t[20] = nan, wrf_data.t[20] = 284.64853


In [9]:
from metpy.units import units
import matplotlib.pyplot as plt
from datetime import datetime as dt
import metpy.calc as mpcalc
from metpy.plots import SkewT
def skewt_plot(data_point, data_type, date_tuple):
    golden = (1. + np.sqrt(5.))/2.
    figsize = (12./golden, 12.)
    fig = plt.figure(figsize=figsize)
    skew = SkewT(fig, rotation=45)
    skew.plot(data_point.p * units.hPa, (data_point.t - 273.15) * units.degC, 'r')
    skew.plot(data_point.p * units.hPa, (data_point.td - 273.15) * units.degC, 'g')
    u, v = mpcalc.wind_components((data_point.wspd / 0.514444) * units.knots, data_point.wdir * units.degrees)
    skew.plot_barbs(data_point.p * units.hPa, u, v)
    skew.ax.set_ylim(1000,100)
    skew.ax.set_xlim(-40,60)
    #skew.plot_dry_adiabats()
    #skew.plot_moist_adiabats()
    #skew.plot_mixing_lines()
    skew.ax.set_title(f'{data_type} Skew T, {dt(*date_tuple).strftime("%Y-%m-%d %H")}z')
    plt.savefig(f'{data_type}_{dt(*date_tuple).strftime("%Y-%m-%d_%H")}z.png')
    plt.show()
    plt.close()

In [11]:
skewt_plot(wrf_data, 'Modeled', (2023,7,21,0,0))

TypeError: loop of ufunc does not support argument 0 of type float which has no callable log10 method

Error in callback <function _draw_all_if_interactive at 0x14f557653560> (for post_execute), with arguments args (),kwargs {}:


TypeError: loop of ufunc does not support argument 0 of type float which has no callable log10 method

TypeError: loop of ufunc does not support argument 0 of type float which has no callable log10 method

<Figure size 741.641x1200 with 1 Axes>

In [32]:
wrf_data.p

masked_array(data=[1009.15356 , 1006.13043 , 1003.1037  , 1000.0562  ,
                    997.0088  ,  992.95996 ,  987.89215 ,  982.86523 ,
                    975.29083 ,  965.20013 ,  955.1112  ,  945.00696 ,
                    934.9106  ,  924.81433 ,  912.1846  ,  897.0452  ,
                    881.92194 ,  866.75653 ,  846.59766 ,  821.3711  ,
                    796.1659  ,  770.96704 ,  743.2577  ,  713.0436  ,
                    682.8374  ,  652.6519  ,  622.5131  ,  592.4098  ,
                    562.3243  ,  532.2526  ,  499.69815 ,  464.66702 ,
                    429.67474 ,  394.722   ,  357.33276 ,  317.51584 ,
                    277.78214 ,  238.12567 ,  200.99731 ,  168.8252  ,
                    139.10657 ,  111.83916 ,   87.00491 ,   64.66303 ,
                     44.840767,   26.802937,   14.936903],
             mask=False,
       fill_value=1e+20,
            dtype=float32)